In [13]:
import numpy as np                      
import itertools                       
import seaborn as sns                    
import matplotlib.pyplot as plt          
import random                            
from datetime import datetime as dt      
from typing import Callable, List, Tuple, Dict
import json
from typing import Dict, Tuple, List, Optional
import csv

DEBUG = True

# Debugging function
def debugger_factory(show_args=True) -> Callable:
    """
    This function creates a debugging wrapper for functions.
    """
    def debugger(func: Callable) -> Callable:
        if DEBUG:   
            def wrapper(*args, **kwargs):
                if show_args:
                    print(f'{func.__name__} was called')  
                t0 = dt.now()                              
                results = func(*args, **kwargs)            
                print(f'{func.__name__} ran for {dt.now()-t0}')  
                return results                             
        else:
            return func                                    
        return wrapper
    return debugger

In [14]:
# We are using half the size of a full deck
HALF_DECK_SIZE = 26

def get_init_deck(half_deck_size: int) -> np.ndarray:
    """
    This function creates an initial decks of 0s (Blacks) and 1s (Reds)
    
    """
    return np.array([0] * half_deck_size + [1] * half_deck_size)

def shuffle_deck(seed: int, deck: np.ndarray) -> np.ndarray:
    """
    This function shuffles a deck using a specific random seed. Shuffles a given deck using a specified seed.
    
    """
    np.random.seed(seed)             # Allows us to store a random seed
    shuffled_deck = deck.copy()      # Making a copy of the deck
    np.random.shuffle(shuffled_deck) # Allows us to shuffle the deck
    return shuffled_deck             # Returns the shuffled deck


@debugger_factory()
def get_n_decks(num_decks: int, num_cards: int = HALF_DECK_SIZE) -> List[Tuple[int, np.ndarray]]:
    """
    This function creates a list of shuffled decks each with unique random seeds. Also, each deck has a tuple containing (seed, deck).
    """
    init_deck = get_init_deck(num_cards)
    decks = []

    for _ in range(num_decks):
        seed = np.random.randint(0, 2**32)  # Generate a unique random seed
        shuffled_deck = shuffle_deck(seed, init_deck)  # Shuffle using the seed
        decks.append((seed, shuffled_deck))  # Store the seed and deck as a tuple

    return decks  # Returns a list of (seed, shuffled deck) pairs


In [26]:
def generate_sequences() -> List[Tuple[int, int, int]]:
    """This function creates all possible combinations of 0s and 1s Generates all possible three-card sequences of 0s and 1s."""
    return list(itertools.product([0, 1], repeat=3))  # This creates all possible combinations of 3-bit sequences 3-bit binary sequence

def generate_1_game(deck: np.ndarray, player1_seq: Tuple[str, str, str], player2_seq: Tuple[str, str, str]) -> Tuple[int, int, int, int]:
    """
    Simulates one game of Penney.
    - Players choose their sequences.
    - Flip cards one at a time until a player's sequence occurs.
    - The player who gets a match is awarded one trick and all previous cards plus the 3 matching cards.
    """
    if player1_seq == player2_seq:
        raise ValueError("Players cannot choose the same sequence.")

    p1_str, p2_str = ''.join(player1_seq), ''.join(player2_seq)
    p1_tricks, p2_tricks = 0, 0
    p1_total_cards, p2_total_cards = 0, 0
    collected_cards = 0  
    current_sequence = []

    for card in deck:
        current_sequence.append(card)
        collected_cards += 1

        
        if len(current_sequence) >= 3:
            trick_str = ''.join(current_sequence[-3:])

            
            if trick_str == p1_str:
                p1_tricks += 1
                p1_total_cards += collected_cards
                collected_cards = 0 
                current_sequence = [] 
                continue

            
            if trick_str == p2_str:
                p2_tricks += 1
                p2_total_cards += collected_cards
                collected_cards = 0  
                current_sequence = []  
                continue

    return p1_tricks, p2_tricks, p1_total_cards, p2_total_cards


def simulate_games(
    num_decks: int = 1000000,
    existing_results: Optional[Dict[Tuple, Dict[str, float]]] = None,
    existing_seeds: Optional[List[int]] = None,
    save_prefix: Optional[str] = None,
    save_to_file: bool = False
) -> Tuple[Dict[Tuple, Dict[str, float]], List[int]]:
    """
    Simulates multiple games of Penney.
    """
    sequences = list(itertools.product('BR', repeat=3))
    results = existing_results if existing_results else {}
    seeds = existing_seeds if existing_seeds else []

    new_seeds = []

    for i in range(num_decks):
        seed = np.random.randint(0, 2**32 - 1)
        new_seeds.append(seed)
        rng = np.random.default_rng(seed)
        deck = rng.choice(['B', 'R'], size=52)

        for player1_seq, player2_seq in itertools.combinations(sequences, 2):
            p1_count, p2_count, p1_cards, p2_cards = generate_1_game(deck, player1_seq, player2_seq)

            for a, b, p1, p2 in [(player1_seq, player2_seq, p1_count, p2_count),
                                 (player2_seq, player1_seq, p2_count, p1_count)]:

                key = (a, b)
                if key not in results:
                    results[key] = {
                        "Player 2 Win % (Trick)": 0,
                        "Player 2 Win % (Total)": 0,
                        "Draw % (Trick)": 0,
                        "Draw % (Total)": 0,
                        "games": 0
                    }

                res = results[key]
                res["Player 2 Win % (Trick)"] += int(p2 > p1)
                res["Player 2 Win % (Total)"] += int(p2_cards > p1_cards)
                res["Draw % (Trick)"] += int(p2 == p1)
                res["Draw % (Total)"] += int(p2_cards == p1_cards)
                res["games"] += 1

    
    for stats in results.values():
        g = stats["games"]
        if g > 0:
            stats["Player 2 Win % (Trick)"] /= g
            stats["Player 2 Win % (Total)"] /= g
            stats["Draw % (Trick)"] /= g
            stats["Draw % (Total)"] /= g

    seeds += new_seeds

    
    if save_to_file and save_prefix:
        with open(f"{save_prefix}_results.json", "w") as f:
            json.dump({str(k): v for k, v in results.items()}, f, indent=2)

        with open(f"{save_prefix}_seeds.csv", "w", newline="") as f:
            writer = csv.writer(f)
            writer.writerow(["seed"])
            writer.writerows([[s] for s in seeds])

        print(f"Saved {len(seeds)} seeds and results to: {save_prefix}_*.json/csv")

    return results, seeds


def visualize_heatmap(results: Dict[Tuple, Dict[str, float]], metrics: List[str], save_path: str = "heatmap"):
    """
    Visualizes win and draw percentages for each scoring method (Tricks & Total Cards).
    """
    sequences = list(itertools.product('BR', repeat=3))
    num_metrics = len(metrics)
    fig, axes = plt.subplots(1, num_metrics, figsize=(8 * num_metrics, 8))
    if num_metrics == 1:
        axes = [axes]
    formatted_labels = [''.join(seq) for seq in sequences]

    for idx, metric in enumerate(metrics):
        matrix = np.zeros((len(sequences), len(sequences)))
        annotations = np.empty((len(sequences), len(sequences)), dtype=object)

        for (p1_seq, p2_seq), stats in results.items():
            i, j = sequences.index(p1_seq), sequences.index(p2_seq)
            win_pct = int(stats.get(metric, 0) * 100)
            draw_metric = "Draw % (Trick)" if "Trick" in metric else "Draw % (Total)"
            draw_pct = int(stats.get(draw_metric, 0) * 100)
            
            annotations[i, j] = f"{win_pct} ({draw_pct})"
            matrix[i, j] = win_pct

        sns.heatmap(
            matrix,
            annot=annotations,
            fmt="",
            xticklabels=formatted_labels,
            yticklabels=formatted_labels,
            cmap="Blues",
            linewidths=0.5,
            annot_kws={"size": 9},
            ax=axes[idx],
            cbar=False
        )
        axes[idx].set_title(f"My Chance of Win (Draw) by {metric}", fontsize=14)
        if idx == 0:  # First heatmap (for Player 2's Win % based on Tricks)
            axes[idx].set_title("My Chance of Win (Draw) \n by Tricks \n N = 1,000,000", fontsize=14)
        elif idx == 1:  # Second heatmap (for Player 2's Win % based on Total Cards)
            axes[idx].set_title("My chance of Win (Draw) \n by Cards \n N = 1,000,000", fontsize=14)
        axes[idx].set_xlabel("Player 2 Choices", fontsize=12)
        axes[idx].set_ylabel("Player 1 Choices", fontsize=12)
        plt.setp(axes[idx].get_xticklabels(), rotation=0, ha="right")
        plt.setp(axes[idx].get_yticklabels(), rotation=0)

    plt.tight_layout()
    filename = f"{save_path}_{metric.replace(' ', '_')}.svg"
    plt.savefig(filename, format="svg")
    print(f"Saved: {filename}")

        # Close the figure to free memory
    plt.close(fig)
    plt.show()


results1, seeds1 = simulate_games(num_decks=1000000, save_prefix="test_aug", save_to_file=True)

# Augment with more decks (put any number of decks in num_decks)
#results2, seeds2 = simulate_games(
#    num_decks=100,
#    existing_results=results1,
#    existing_seeds=seeds1,
#    save_prefix="test_aug",
#    save_to_file=True
#)

visualize_heatmap(results1, ["Player 2 Win % (Trick)", "Player 2 Win % (Total)"])


Saved 1000000 seeds and results to: test_aug_*.json/csv
Saved: heatmap_Player_2_Win_%_(Total).svg
